In [ ]:
from buttermilk.utils.nb import *
import nest_asyncio
from typing import cast

# Apply nest_asyncio to handle potential event loop issues in notebooks
nest_asyncio.apply()
import chromadb

# Add this to the first cell or a new cell before getting the collection
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel
import asyncio
import nest_asyncio
from typing import cast

# Apply nest_asyncio to handle potential event loop issues in notebooks
nest_asyncio.apply()

PERSIST_DIR = "/home/nic/data/prosocial_zot/files"
COLLECTION_NAME = "prosocial_zot"
MODEL_NAME = "text-embedding-large-exp-03-07"  # From your vector.py
TASK_FOR_QUERY = "RETRIEVAL_QUERY"  # Use RETRIEVAL_QUERY for query embedding
DIMENSIONALITY = 3072  # From your vector.py
embedding_model = TextEmbeddingModel.from_pretrained(MODEL_NAME)

client = chromadb.PersistentClient(path=PERSIST_DIR)

class VertexAIEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input) -> Embeddings:
        kwargs = dict(output_dimensionality=DIMENSIONALITY, auto_truncate=False)
        results = embedding_model.get_embeddings(texts=input)
        return [cast(list[float], r.values) for r in results]

vertex_ef = VertexAIEmbeddingFunction()
collection = client.get_collection(name=COLLECTION_NAME, embedding_function=vertex_ef)
print(f"Retrieved collection '{COLLECTION_NAME}' with Vertex AI embedding function.")


In [ ]:
# Define your query text
query_text = (
    "bias metrics in nlp are sensitive to power and hierarchy and positionality"
)

# Query the collection
results = collection.query(
    query_texts=[query_text],
    n_results=20,
    include=[
        "documents",
        "metadatas",
        "distances",
    ],  # Include documents and optionally metadata/distances
)


In [56]:
import pydantic


class RefResult(pydantic.BaseModel):
    id: str
    full_text: str
    uri: str | None
    chunk_index: int
    citation: str
    document_id: str
    document_title: str
    doi_or_url: str
    _extra_zotero_data: str


def mk_result(results, index) -> RefResult:
    result = {
        x: results[x][0][index]
        for x in [
            "ids",
            "metadatas",
            "documents",
        ]
        if results[x]
    }
    meta = {k: v for k, v in result["metadatas"].items() if k in RefResult.model_fields}
    return RefResult(
        id=result["ids"],
        full_text=result["documents"],
        uri=result.get("uri", ""),
        **meta,
    )


In [58]:
console.print_json(data=mk_result(results, 3).model_dump())


{
  "id": "3459810b-b699-48ed-bd1d-91592571438d",
  "full_text": "EU High-Level Expert Group on AI. (2019). Ethics guidelines for trustworthy AI. https://ec.europa.eu/newsroom/\n\ndae/document.cfm?doc_id=60419\n\n\f14 of 19\n\nHOVY and PRABHUMOYE\n\nFlek, L. (2020). Returning the N to NLP: Towards contextually personalized classification models. Proceedings of \nthe 58th Annual Meeting of the Association for Computational Linguistics, 7828–7838. Online: Association for \nComputational Linguistics. https://www.aclweb.org/anthology/2020.acl-main.700\n\nFont, J. E., & Costa-jussà, M. R. (2019). Equalizing gender bias in neural machine translation with word embed-\ndings techniques. Proceedings of the First Workshop on Gender Bias in Natural Language Processing, 147–154. \nFlorence, Italy: Association for Computational Linguistics. https://www.aclweb.org/anthology/W19-3821\nFornaciari, T., Uma, A., Paun, S., Plank, B., Hovy, D., & Poesio, M. (2021). Beyond black & white: Leveraging an-\nnotator disagreement via soft-label multi-task learning. Proceedings of the 2021 Conference of the North Amer-\nican  Chapter  of  the  Association  for  Computational  Linguistics:  Human  Language  Technologies,  2591–2597. \nOnline: Association for Computational Linguistics. https://www.aclweb.org/anthology/2021.naacl-main.204\nFort, K., Adda, G., & Cohen, K. B. (2011). Last words: Amazon Mechanical Turk: Gold mine or coal mine? Com-\n\nputational Linguistics, 37, 413–420. https://www.aclweb.org/anthology/J11-2010\n\nFriedler, S. A., Scheidegger, C., & Venkatasubramanian, S. (2021). The (im) possibility of fairness: Different value \nsystems require different mechanisms for fair decision making. Communications of the ACM, 64, 136–143.\nFromreide, H., Hovy, D., & Søgaard, A. (2014). Crowdsourcing and annotating NER for Twitter #drift. Proceedings \nof  the  Ninth  International  Conference  on  Language  Resources  and  Evaluation  (LREC’14),  2544–2547.  Rey-\nkjavik, Iceland: European Language Resources Association (ELRA). http://www.lrec-conf.org/proceedings/\nlrec2014/pdf/421_Paper.pdf\n\nGarg, N., Schiebinger, L., Jurafsky, D., & Zou, J. (2018). Word embeddings quantify 100 years of gender and ethnic \n\nstereotypes. Proceedings of the National Academy of Sciences, 115, E3635–E3644.\n\nGarimella, A., Banea, C., Hovy, D., & Mihalcea, R. (2019). Women's syntactic resilience and men's grammatical \nluck: Gender-bias in part-of-speech tagging and dependency parsing. Proceedings of the 57th Annual Meeting \nof the Association for Computational Linguistics, 3493–3498. Florence, Italy: Association for Computational \nLinguistics. https://www.aclweb.org/anthology/P19-1339\n\nGoldstein, D. G., & Gigerenzer, G. (2002). Models of ecological rationality: The recognition heuristic. Psychological \n\nReview, 109, 75–90.\n\nGonen, H., & Goldberg, Y. (2019). Lipstick on a pig: Debiasing methods cover up systematic gender biases in word \nembeddings but do not remove them. Proceedings of the 2019 Conference of the North American Chapter of the \nAssociation for Computational Linguistics: Human Language Technologies, Volume 1 (Long and Short Papers), \n609–614. https://www.aclweb.org/anthology/N19-1061\n\nGrouin, C., Griffon, N., & Névéol, A. (2015). Is it possible to recover personal health information from an auto-\nmatically de-identified corpus of French EHRs? Proceedings of the Sixth International Workshop on Health \nText Mining and Information Analysis, 31–39. Lisbon, Portugal: Association for Computational Linguistics. \nhttps://www.aclweb.org/anthology/W15-2604\n\nGururangan, S., Swayamdipta, S., Levy, O., Schwartz, R., Bowman, S. R., & Smith, N. A. (2018). Annotation arti-\n\nfacts in natural language inference data. In NAACL-HLT (2).\n\nHarwell, D. (2018). The accent gap. Why some accents don’t work on Alexa or Google Home. The Washington Post. \n\nhttps://www.washingtonpost.com/graphics/2018/business/alexa-does-not-understand-your-accent/\n\nHaven

In [ ]:
index = 3

console.print_json(data=result)


{
  "ids": "3459810b-b699-48ed-bd1d-91592571438d",
  "metadatas": {
    "chunk_index": 8,
    "citation": "Hovy, D., & Prabhumoye, S. (2021). Five sources of bias in natural language processing. *Language and Linguistics Compass*, *15*(7), e12432. https://doi.org/10.1111/lnc3.12432",
    "document_id": "8S6C9EF9",
    "document_title": "Five sources of bias in natural language processing",
    "doi_or_url": "10.1111/lnc3.12432",
    "zotero_data": "{\"key\": \"8S6C9EF9\", \"version\": 37517, \"itemType\": \"journalArticle\", \"title\": \"Five sources of bias in natural language processing\", \"creators\": [{\"creatorType\": \"author\", \"firstName\": \"Dirk\", \"lastName\": \"Hovy\"}, {\"creatorType\": \"author\", \"firstName\": \"Shrimai\", \"lastName\": \"Prabhumoye\"}], \"abstractNote\": \"Recently, there has been an increased interest in demographically grounded bias in natural language processing (NLP) applications. Much of the recent work has focused on describing bias and providing an overview of bias in a larger context. Here, we provide a simple, actionable summary of this recent work. We outline five sources where bias can occur in NLP systems: (1) the data, (2) the annotation process, (3) the input representations, (4) the models, and finally (5) the research design (or how we conceptualize our research). We explore each of the bias sources in detail in this article, including examples and links to related work, as well as potential counter-measures.\", \"publicationTitle\": \"Language and Linguistics Compass\", \"volume\": \"15\", \"issue\": \"8\", \"pages\": \"e12432\", \"date\": \"2021\", \"series\": \"\", \"seriesTitle\": \"\", \"seriesText\": \"\", \"journalAbbreviation\": \"\", \"language\": \"en\", \"DOI\": \"10.1111/lnc3.12432\", \"ISSN\": \"1749-818X\", \"shortTitle\": \"\", \"url\": \"https://onlinelibrary.wiley.com/doi/abs/10.1111/lnc3.12432\", \"accessDate\": \"2024-07-04T12:35:01Z\", \"archive\": \"\", \"archiveLocation\": \"\", \"libraryCatalog\": \"Wiley Online Library\", \"callNumber\": \"\", \"rights\": \"© 2021 The Authors. Language and Linguistics Compass published by John Wiley & Sons Ltd.\", \"extra\": \"_eprint: https://onlinelibrary.wiley.com/doi/pdf/10.1111/lnc3.12432\", \"tags\": [], \"collections\": [\"NQDTJJDN\"], \"relations\": {}, \"dateAdded\": \"2024-07-04T12:35:02Z\", \"dateModified\": \"2024-07-04T12:35:02Z\"}"
  },
  "documents": "EU High-Level Expert Group on AI. (2019). Ethics guidelines for trustworthy AI. https://ec.europa.eu/newsroom/\n\ndae/document.cfm?doc_id=60419\n\n\f14 of 19\n\nHOVY and PRABHUMOYE\n\nFlek, L. (2020). Returning the N to NLP: Towards contextually personalized classification models. Proceedings of \nthe 58th Annual Meeting of the Association for Computational Linguistics, 7828–7838. Online: Association for \nComputational Linguistics. https://www.aclweb.org/anthology/2020.acl-main.700\n\nFont, J. E., & Costa-jussà, M. R. (2019). Equalizing gender bias in neural machine translation with word embed-\ndings techniques. Proceedings of the First Workshop on Gender Bias in Natural Language Processing, 147–154. \nFlorence, Italy: Association for Computational Linguistics. https://www.aclweb.org/anthology/W19-3821\nFornaciari, T., Uma, A., Paun, S., Plank, B., Hovy, D., & Poesio, M. (2021). Beyond black & white: Leveraging an-\nnotator disagreement via soft-label multi-task learning. Proceedings of the 2021 Conference of the North Amer-\nican  Chapter  of  the  Association  for  Computational  Linguistics:  Human  Language  Technologies,  2591–2597. \nOnline: Association for Computational Linguistics. https://www.aclweb.org/anthology/2021.naacl-main.204\nFort, K., Adda, G., & Cohen, K. B. (2011). Last words: Amazon Mechanical Turk: Gold mine or coal mine? Com-\n\nputational Linguistics, 37, 413–420. https://www.aclweb.org/anthology/J11-2010\n\nFriedler, S. A., Scheidegger, C., & Venkatasubramanian, S. (2021). The (im) possibility of fairness: Different value \nsyste

In [60]:
results.keys()


dict_keys(['ids', 'embeddings', 'documents', 'uris', 'data', 'metadatas', 'distances', 'included'])